In [ ]:
import scipy.io.wavfile as wav
import numpy as np
from scipy.signal import lfilter
import random
import matplotlib.pyplot as plt


# Functions

In [ ]:
def convm(x, p):
  N = len(x)+2*p-2;
  x = x[:]
  xpad = np.array([[np.zeros(p-1, 1)],[x],[np.zeros(p-1,1)]])
  for i in range(1,p):
    X[:, i] = xpad[p-i+1:N-i+1]
  return X


In [ ]:
def covar(x, p):
  x = x[:]
  m = len(x)
  x = x - np.ones(m, 1) * (sum(x)/m)
  R = convm(x, p).getH() * convm(x, p) / (m-1)
  return R

In [ ]:
def speechProduct(inputSignal, generator, p):
  Rxp = covar(inputSignal, p+1)
  Rx = Rxp[1:p, 1:p]
  rx = Rx[2:, 1]
  rx = [rx.getH(), Rxp[-1, 1]]
  w = rx * np.linalg.inv(Rx)

  G2 = Rx[1, 1] - sum(np.multiply( w , Rxp[1, 2:]))
  G = np.sqrt(G2)

  w = w * -1
  ak = [1,w]
  #% H_z = filter(G, ak, inputSignal) 

  outputSignal = lfilter(G, ak, generator)
  return outputSignal
  

In [ ]:
def rect(T):
    """create a centered rectangular pulse of width $T"""
    return lambda t: (-T/2 <= t) & (t < T/2)

def pulse_train(t, at, shape):
    """create a train of pulses over $t at times $at and shape $shape"""
    return np.sum(shape(t - at[:,np.newaxis]), axis=0)


# Combine Signals

In [ ]:
#Signal Sh
[samplRate_1,inputSignal_1] = wav.read('Sh.wav')
[m1, n1] = inputSignal_1.shape[0],inputSignal_1.shape[1]
t1 = np.linspace(0,1,m1-1)
d1 = np.linspace(0,1,m1-1)
# out1 = speechProduct(inputSignal_1, pulse_train(t1, d1, 'tripuls', 0.1, 0), 1000);
out1 = speechProduct(inputSignal_1, pulse_train(t1, d1,  rect(0.1)   ), 1000)

#Signal I
[inputSignal_2, samplRate_2] = wav.read('I.wav')
[m2, n2] = inputSignal_2.shape[0],inputSignal_2.shape[1]
out2 = speechProduct(inputSignal_2, np.random.randn(1, m2), 300)

#Signal R
[inputSignal_3, samplRate_3] = wav.read('R.wav')
[m3, n3] = inputSignal_3.shape[0],inputSignal_3.shape[1]
out3 = speechProduct(inputSignal_3, np.random.randn(1, m2), 300)

#Signal I
[inputSignal_4, samplRate_4] = wav.read('I.wav')
[m4, n4] = inputSignal_4.shape[0],inputSignal_4.shape[1]
t4 = np.linspace(0,1,m4-1)
d4 = np.linspace(0,1,m4-1)
out4 = speechProduct(inputSignal_4, pulse_train(t4, d4,  rect(0.1) ), 1000)

#all Signal
out = np.zeros(1, m1+m2+m3+m4)
out[1 : m1] = out1
out[m1+1 : m1+m2] = out2
out[m1+m2+1 : m1+m2+m3 ] = out3
out[m1+m2+m3+1 : m1+m2+m3+m4] = out4
plt.figure()
plt.plot(out)
wav.write('shiri.wav',sound(out, samplRate_1))

